In [1]:
!nvidia-smi

Thu Jan  4 13:17:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
!pip install -q transformers[sentencepiece] ohmeow-blurr fastbook fastai nbdev

In [ ]:
import torch
from fastai.text.all import *
from blurr.text.data.all import *
from transformers import AutoConfig
from blurr.text.modeling.all import *
from transformers import AutoModelForSequenceClassification

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/keyword_generation/

In [ ]:
df = pd.read_csv("data/papers_final_data.csv")

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
type(df.iloc[0]['final_keywords'])

In [ ]:
for index in range(len(df)):
  df.iloc[index]['final_keywords'] = [ item.replace(" '","").replace("'","").lower() for item in re.split(",", df.iloc[index]['final_keywords'].replace("[","").replace("]","")) ]

In [ ]:
type(df.iloc[0]['final_keywords'])

In [ ]:
available_keywords = {}

keywords_list = df['final_keywords'].to_list()

for item in range(len(keywords_list)):
  for keyword in keywords_list[item]:
    if keyword in available_keywords.keys():
      available_keywords[keyword] += 1
    else:
      available_keywords[keyword] = 1

In [ ]:
available_keywords

In [ ]:
encoding_path = "encoding"

In [ ]:
encode_keywords = { key: idx for idx, (key, value) in enumerate(available_keywords.items())}
with open(f"{encoding_path}/encoded_keywords.json", "w") as fp:
  json.dump(encode_keywords, fp)

In [ ]:
encode_keywords

In [ ]:
categorical_keywords_list = []

for keywords in keywords_list:
  categorical_list = [0]*len(encode_keywords)
  for keyword in keywords:
    keyword_index = encode_keywords[keyword]
    categorical_list[keyword_index] = 1
  categorical_keywords_list.append(categorical_list)

In [ ]:
df['encoded_keywords'] = categorical_keywords_list

In [ ]:
df.shape

In [ ]:
df.tail()

In [ ]:
labels = list(encode_keywords.keys())

In [ ]:
labels

**RoBERTa** -> roberta-base <br/>
**BERT** -> bert-base-uncased<br/>
**XLNet** -> xlnet-base-cased

In [ ]:
# model_name = "roberta-base"
# model_name = "bert-base-uncased"
model_name = "xlnet-base-uncased"

model_cls = AutoModelForSequenceClassification

config = AutoConfig.from_pretrained(model_name)

config.num_labels = len(labels)

hf_arch, hf_config, hf_tokenizer, hf_model = get_hf_objects(model_name, model_cls = model_cls, config = config)

hf_model.config.problem_type = "multi_label_classification"

In [ ]:
hf_model

In [ ]:
blocks = (
    TextBlock(hf_arch, hf_config, hf_tokenizer, hf_model),
    MultiCategoryBlock(encoded=True, vocab=labels)
    )

dblock = DataBlock(
    blocks=blocks,
    get_x = ColReader('abstracts'),
    get_y = ColReader('encoded_keywords'),
    splitter = RandomSplitter(valid_pct = 0.1, seed = 42)
)


In [ ]:
batch_size = 16

In [ ]:
dls = dblock.dataloaders(df, bs=batch_size)

In [ ]:
dataloaders_path = "dataloaders"

In [ ]:
torch.save(dls, f"{dataloaders_path}/xlnet_base_0.003.pkl")

In [ ]:
dls.show_batch(dataloaders = dls, max_n = 4, trunc_at = 512)